<a href="https://colab.research.google.com/github/Phronesis255/RS-Tagging/blob/main/RS_Content_Tagging2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # prompt: upload and import a csv with only one key "URL" and save it as the list "top_urls=[]"

# import pandas as pd
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

# df = pd.read_csv(fn)
# top_urls = df['URL'].tolist()
# top_urls


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_name = '/content/drive/My Drive/RS-Post-URL-Input.csv'

df = pd.read_csv(file_name)
top_urls = df['URL'].tolist()
top_urls


Mounted at /content/drive


['https://www.runsensible.com/blog/wet-signature-vs-electronic-signature/',
 'https://www.runsensible.com/blog/model-law-firm-hierarchy-structure/',
 'https://www.runsensible.com/blog/fairness-justice-role-judicial-codes-of-conduct-legal-practice/',
 'https://www.runsensible.com/blog/texting-clients-ethical-guide-for-lawyers/',
 'https://www.runsensible.com/blog/5-cost-saving-strategies-legal-technology/',
 'https://www.runsensible.com/blog/personal-injury-lawyer-commercial/',
 'https://www.runsensible.com/blog/saas-lawyer-legal-essentials-practices/',
 'https://www.runsensible.com/blog/year-end-closing-law-firm-start-new-year/',
 'https://www.runsensible.com/blog/software-lawyers-law-firms-use/',
 'https://www.runsensible.com/blog/top-10-myths-about-solo-attorneys/',
 'https://www.runsensible.com/blog/6-benefits-client-portal-law-firms/',
 'https://www.runsensible.com/blog/insight-prenuptial-agreements/',
 'https://www.runsensible.com/blog/dropbox-for-lawyers-document-storage/',
 'htt

In [ ]:
file_name_tags = '/content/drive/My Drive/RS-Post-Tags.csv'

df = pd.read_csv(file_name_tags)
predefined_topics = df['topics'].tolist()
predefined_topics

['Civil Litigation',
 'Real Estate Law',
 'Business Law',
 'Bankruptcy Law',
 'Personal Injury Law',
 'Immigration Law',
 'Estate Planning Law',
 'Client Intake',
 'Document Automation & E-Signature',
 'Client Relationship Management (CRM)',
 'Cloud Phone System',
 'Calendaring',
 'Legal Case Management',
 'Workflow and Email Automation',
 'Document Management',
 'RunSensible Drive',
 'Task Management',
 'Time & Expense Tracking',
 'Invoicing',
 'Legal Accounting',
 'Trust Accounting',
 'Online Payments']

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from transformers import pipeline
import pandas as pd


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from transformers import pipeline
import pandas as pd

# Function to Extract Titles and Content from URLs
def extract_content_from_url(url, retries=2, timeout=5):
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                title = soup.title.string.strip() if soup.title and soup.title.string else "No Title"
                content = soup.get_text(separator=' ').strip()
                return title, content
        except requests.RequestException:
            pass
        time.sleep(2)
    return None, None


# Ensure only the top 60 URLs are processed
top_urls = top_urls[:70]

# Lists to store extracted data
successful_urls = []
titles = []
contents = []

# Step 1: Extract Titles and Content from URLs
print("Extracting titles and content...")
for idx, url in enumerate(top_urls):
    print(f"Retrieving content from {url}...")
    title, content = extract_content_from_url(url)
    time.sleep(1)  # Delay between requests to avoid rate-limiting
    if title is None:
        title = "No Title"

    if content:
        successful_urls.append(url)
        titles.append(title)
        contents.append(content)

print(f"Extracted {len(successful_urls)} URLs successfully.")

# Step 2: Save Extracted Data to a CSV
data = pd.DataFrame({"URL": successful_urls, "Title": titles, "Content": contents})
data.to_csv('content_data.csv', index=False)
print("Content data saved to 'content_data.csv'.")

# Step 3: Define Predefined Topics for ZSC
# predefined_topics = [
#     "Data Privacy",
#     "Client Intake",
#     "Document Automation",
#     "Real Estate Law",
#     "Family Law",
#     "Personal Injury Law",
#     # Add additional topics as needed
# ]

# Step 4: Initialize Zero-Shot Classification Pipeline
print("Initializing Zero-Shot Classification model...")
zsc = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Step 5: Perform Zero-Shot Classification on Content
print("Classifying content...")
classified_data = []
for idx, content in enumerate(contents):
    print(f"Classifying content from URL {idx + 1}/{len(contents)}: {successful_urls[idx]}")
    # Shorten content to fit model input size if necessary (truncate to 1024 tokens)
    truncated_content = content[:512]  # Adjust truncation length based on model token limit
    result = zsc(
        truncated_content,
        candidate_labels=predefined_topics,
        multi_label=True  # Enables multi-label classification
    )
    classified_data.append({
        "Title": titles[idx],
        "URL": successful_urls[idx],
        "Predicted_Topics": ", ".join(result["labels"]),
        "Confidence_Scores": ", ".join([f"{score:.2f}" for score in result["scores"]])
    })

# Step 6: Save Classified Content with Predictions
classified_content_df = pd.DataFrame(classified_data)
classified_content_df.to_csv('classified_content.csv', index=False)
print("Classification results saved to 'classified_content.csv'.")
from google.colab import files
files.download('classified_content.csv')

NameError: name 'top_urls' is not defined

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Function to Extract Titles and Content from URLs
def extract_title_from_url(url, retries=2, timeout=5):
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                title = soup.title.string.strip() if soup.title and soup.title.string else "No Title"
                return title
        except requests.RequestException:
            pass
        time.sleep(2)
    return None, None

max = 60

top_urls = top_urls[:max]
# Lists to store extracted data
successful_urls = []
titles = []

# Step 1: Extract Titles and Content from URLs
print("Extracting titles and content...")
for idx, url in enumerate(top_urls):
    print(f"Retrieving content from {url}...")
    title = extract_title_from_url(url)
    time.sleep(1)
    if title is None:
        title = "No Title"

    successful_urls.append(url)
    titles.append(title)
    if len(titles) >= max:  # Max content limit for testing
        break

print(f"Extracted {len(successful_urls)} URLs successfully.")

# Step 2: Save Extracted Titles and URLs to a CSV
data = pd.DataFrame({"URL": successful_urls, "Title": titles})
data.to_csv('titles_and_urls.csv', index=False)
print("Titles and URLs saved to 'titles_and_urls.csv'.")
files.download('titles_and_urls.csv')
# Step 3: Define Predefined Topics for ZSC
# predefined_topics = [
#     "Data Privacy",
#     "Client Intake",
#     "Document Automation",
#     "Real Estate Law",
#     "Family Law",
#     "Personal Injury Law",
#     # Add additional topics as needed
# ]

# Step 4: Initialize Zero-Shot Classification Pipeline
print("Initializing Zero-Shot Classification model...")
zsc = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Step 5: Perform Zero-Shot Classification on Titles
print("Classifying titles...")
classified_data = []
for idx, title in enumerate(titles):
    print(f"Classifying title {idx + 1}/{len(titles)}: {title}")
    result = zsc(
        title,
        candidate_labels=predefined_topics,
        multi_label=False  # Enables multi-label classification
    )
    classified_data.append({
        "Title": title,
        "URL": successful_urls[idx],
        "Predicted_Topics": ", ".join(result["labels"]),
        "Confidence_Scores": ", ".join([f"{score:.2f}" for score in result["scores"]])
    })

# Step 6: Save Classified Titles with Predictions
classified_title_df = pd.DataFrame(classified_data)
classified_title_df.to_csv('classified_titles.csv', index=False)
print("Classification results saved to 'classified_titles.csv'.")
files.download('classified_titles.csv')

Extracting titles and content...
Retrieving content from https://www.runsensible.com/blog/wet-signature-vs-electronic-signature/...
Retrieving content from https://www.runsensible.com/blog/model-law-firm-hierarchy-structure/...
Retrieving content from https://www.runsensible.com/blog/fairness-justice-role-judicial-codes-of-conduct-legal-practice/...
Retrieving content from https://www.runsensible.com/blog/texting-clients-ethical-guide-for-lawyers/...
Retrieving content from https://www.runsensible.com/blog/5-cost-saving-strategies-legal-technology/...
Retrieving content from https://www.runsensible.com/blog/personal-injury-lawyer-commercial/...
Retrieving content from https://www.runsensible.com/blog/saas-lawyer-legal-essentials-practices/...
Retrieving content from https://www.runsensible.com/blog/year-end-closing-law-firm-start-new-year/...
Retrieving content from https://www.runsensible.com/blog/software-lawyers-law-firms-use/...
Retrieving content from https://www.runsensible.com/b

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Initializing Zero-Shot Classification model...
Classifying titles...
Classifying title 1/60: Wet Signature vs. Electronic Signature: Key Differences and When to Use Each - RunSensible
Classifying title 2/60: What Is the Model Law Firm Hierarchy and Structure? - RunSensible
Classifying title 3/60: Ensuring Fairness and Justice: The Role of Judicial Codes of Conduct in Legal Practice - RunSensible
Classifying title 4/60: The Do's and Don'ts of Texting Clients: An Ethical Guide for Lawyers - RunSensible
Classifying title 5/60: 5 Cost-Saving Strategies: How Legal Departments Leverage Technology - RunSensible
Classifying title 6/60: 16 Personal Injury Lawyer Commercial Examples - RunSensible
Classifying title 7/60: Becoming a SaaS Lawyer: Legal Essentials and Best Practices - RunSensible
Classifying title 8/60: Year-End Closing Questions to Help Your Law Firm Start Next Year Strong - RunSensible
Classifying title 9/60: Essential Software Lawyers and Law Firms Use - RunSensible
Classifying t

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

classified_title_df = pd.read_csv(fn)
#predefined_topics = df['topics'].tolist()
classified_title_df

Saving classified_titles (1).csv to classified_titles (1).csv
User uploaded file "classified_titles (1).csv" with length 8587 bytes


,Title,URL,Predicted_Topics,Confidence_Scores
0,Wet Signature vs. Electronic Signature: Key Di...,https://www.runsensible.com/blog/wet-signature...,"Document Automation, Client Intake, Data Priva...","0.33, 0.27, 0.19, 0.08, 0.06, 0.06"
1,What Is the Model Law Firm Hierarchy and Struc...,https://www.runsensible.com/blog/model-law-fir...,"Client Intake, Document Automation, Data Priva...","0.44, 0.20, 0.11, 0.10, 0.09, 0.06"
2,Ensuring Fairness and Justice: The Role of Jud...,https://www.runsensible.com/blog/fairness-just...,"Client Intake, Document Automation, Personal I...","0.30, 0.27, 0.13, 0.12, 0.09, 0.09"
3,The Do's and Don'ts of Texting Clients: An Eth...,https://www.runsensible.com/blog/texting-clien...,"Client Intake, Data Privacy, Document Automati...","0.25, 0.24, 0.20, 0.13, 0.12, 0.07"
4,5 Cost-Saving Strategies: How Legal Department...,https://www.runsensible.com/blog/5-cost-saving...,"Client Intake, Document Automation, Data Priva...","0.34, 0.26, 0.12, 0.11, 0.11, 0.06"
5,16 Personal Injury Lawyer Commercial Examples ...,https://www.runsensible.com/blog/personal-inju...,"Personal Injury Law, Client Intake, Document A...","0.96, 0.02, 0.01, 0.01, 0.01, 0.00"
6,Becoming a SaaS Lawyer: Legal Essentials and B...,https://www.runsensible.com/blog/saas-lawyer-l...,"Client Intake, Document Automation, Data Priva...","0.41, 0.27, 0.13, 0.07, 0.06, 0.06"
7,Year-End Closing Questions to Help Your Law Fi...,https://www.runsensible.com/blog/year-end-clos...,"Client Intake, Document Automation, Data Priva...","0.25, 0.24, 0.15, 0.14, 0.13, 0.10"
8,Essential Software Lawyers and Law Firms Use -...,https://www.runsensible.com/blog/software-lawy...,"Client Intake, Document Automation, Data Priva...","0.25, 0.24, 0.17, 0.14, 0.11, 0.09"
9,Top 10 Myths About Solo Attorneys - RunSensible,https://www.runsensible.com/blog/top-10-myths-...,"Client Intake, Document Automation, Personal I...","0.39, 0.24, 0.11, 0.11, 0.09, 0.06"


In [ ]:
import pandas as pd
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

classified_content_df = pd.read_csv(fn)
#predefined_topics = df['topics'].tolist()
classified_content_df

Saving classified_content (1).csv to classified_content (1) (1).csv
User uploaded file "classified_content (1) (1).csv" with length 43796 bytes


,Title,URL,Predicted_Topics,Confidence_Scores
0,Wet Signature vs. Electronic Signature: Key Di...,https://www.runsensible.com/blog/wet-signature...,"Document Automation & E-Signature, Client Inta...","0.97, 0.83, 0.70, 0.23, 0.22, 0.16, 0.13, 0.12..."
1,What Is the Model Law Firm Hierarchy and Struc...,https://www.runsensible.com/blog/model-law-fir...,"Document Automation & E-Signature, Client Inta...","0.89, 0.83, 0.61, 0.44, 0.44, 0.37, 0.35, 0.25..."
2,Ensuring Fairness and Justice: The Role of Jud...,https://www.runsensible.com/blog/fairness-just...,"Client Intake, Client Relationship Management ...","0.49, 0.13, 0.12, 0.10, 0.08, 0.07, 0.07, 0.06..."
3,The Do's and Don'ts of Texting Clients: An Eth...,https://www.runsensible.com/blog/texting-clien...,"Client Intake, Document Automation & E-Signatu...","0.69, 0.62, 0.33, 0.18, 0.17, 0.17, 0.10, 0.09..."
4,5 Cost-Saving Strategies: How Legal Department...,https://www.runsensible.com/blog/5-cost-saving...,"Document Automation & E-Signature, Client Inta...","0.72, 0.68, 0.22, 0.20, 0.19, 0.15, 0.15, 0.13..."
5,16 Personal Injury Lawyer Commercial Examples ...,https://www.runsensible.com/blog/personal-inju...,"Document Automation & E-Signature, Personal In...","0.82, 0.75, 0.42, 0.32, 0.17, 0.12, 0.09, 0.06..."
6,Becoming a SaaS Lawyer: Legal Essentials and B...,https://www.runsensible.com/blog/saas-lawyer-l...,"Document Automation & E-Signature, Client Inta...","0.81, 0.67, 0.43, 0.34, 0.32, 0.23, 0.22, 0.17..."
7,Year-End Closing Questions to Help Your Law Fi...,https://www.runsensible.com/blog/year-end-clos...,"Document Automation & E-Signature, Client Inta...","0.60, 0.54, 0.21, 0.16, 0.14, 0.12, 0.12, 0.10..."
8,Essential Software Lawyers and Law Firms Use -...,https://www.runsensible.com/blog/software-lawy...,"Document Automation & E-Signature, Client Inta...","0.88, 0.75, 0.53, 0.35, 0.25, 0.16, 0.15, 0.11..."
9,Top 10 Myths About Solo Attorneys - RunSensible,https://www.runsensible.com/blog/top-10-myths-...,"Client Relationship Management (CRM), Document...","0.96, 0.95, 0.62, 0.40, 0.15, 0.15, 0.11, 0.10..."


In [ ]:
# prompt: compare the first confidence score for each URL in classified_title_df vs that in classified_content_df and save the topics of with a higher score. Let's call the new data frame classified_integrated.

import pandas as pd

# Load the classified dataframes
# Function to extract the first confidence score
def get_first_confidence_score(score_str):
    try:
        return float(score_str.split(',')[0])
    except (ValueError, IndexError, AttributeError):
        return 0  # Handle cases with missing or invalid scores


# Apply the function to create new columns with the first confidence scores
classified_title_df['First_Confidence_Score'] = classified_title_df['Confidence_Scores'].apply(get_first_confidence_score)
classified_content_df['First_Confidence_Score'] = classified_content_df['Confidence_Scores'].apply(get_first_confidence_score)

# Merge the dataframes on the 'URL' column
merged_df = pd.merge(classified_title_df, classified_content_df, on='URL', suffixes=('_title', '_content'))
print(merged_df.head())

# Create a new DataFrame 'classified_integrated' with the higher confidence score
classified_integrated = pd.DataFrame()
classified_integrated['URL'] = merged_df['URL']
classified_integrated['Predicted_Topics'] = ''

for index, row in merged_df.iterrows():
    if row['First_Confidence_Score_title'] >= row['First_Confidence_Score_content']:
        classified_integrated.loc[index, 'Predicted_Topics'] = row['Predicted_Topics_title']
    else:
        classified_integrated.loc[index, 'Predicted_Topics'] = row['Predicted_Topics_content']

# Save the new DataFrame to a CSV file
classified_integrated.to_csv('classified_integrated.csv', index=False)
print("Integrated classification results saved to 'classified_integrated.csv'")
files.download('classified_integrated.csv')

                                         Title_title  \
0  Wet Signature vs. Electronic Signature: Key Di...   
1  What Is the Model Law Firm Hierarchy and Struc...   
2  Ensuring Fairness and Justice: The Role of Jud...   
3  The Do's and Don'ts of Texting Clients: An Eth...   
4  5 Cost-Saving Strategies: How Legal Department...   

                                                 URL  \
0  https://www.runsensible.com/blog/wet-signature...   
1  https://www.runsensible.com/blog/model-law-fir...   
2  https://www.runsensible.com/blog/fairness-just...   
3  https://www.runsensible.com/blog/texting-clien...   
4  https://www.runsensible.com/blog/5-cost-saving...   

                              Predicted_Topics_title  \
0  Document Automation, Client Intake, Data Priva...   
1  Client Intake, Document Automation, Data Priva...   
2  Client Intake, Document Automation, Personal I...   
3  Client Intake, Data Privacy, Document Automati...   
4  Client Intake, Document Automation, Data Pr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt: I want to compare the first confidence score for each URL in classified_title_df vs that in classified_content_df and save the topics of with a higher score. Let's call the new data frame classified_integrated. I wrote this code, but the data it creates is just a column of titles and a column of a all predicted topics added together. I want these columns for classified_integrated: title, URL, most confident topics, prediction confidence scores for those topics. Here is an example, if the a URL has "client intake" as topic with a prediction confidence score of 0.7 based on its title and it's classified under "document management" with a prediction confidence score of 0.85 based on its content, I want its topic and confidence score to be taken from classified_content (the more confident prediction).
# import pandas as pd
# # Load the classified dataframes
# # Function to extract the first confidence score
# def get_first_confidence_score(score_str):
#     try:
#         return float(score_str.spli

# Load the classified dataframes
# Function to extract the first confidence score
def get_first_confidence_score(score_str):
    try:
        return float(score_str.split(',')[0])
    except (ValueError, IndexError, AttributeError):
        return 0  # Handle cases with missing or invalid scores


# Apply the function to create new columns with the first confidence scores
classified_title_df['First_Confidence_Score'] = classified_title_df['Confidence_Scores'].apply(get_first_confidence_score)
classified_content_df['First_Confidence_Score'] = classified_content_df['Confidence_Scores'].apply(get_first_confidence_score)

# Merge the dataframes on the 'URL' column
merged_df = pd.merge(classified_title_df, classified_content_df, on='URL', suffixes=('_title', '_content'))

# Create a new DataFrame 'classified_integrated'
classified_integrated = pd.DataFrame()
classified_integrated['URL'] = merged_df['URL']
classified_integrated['Title'] = merged_df['Title_title'] #Use title from title df
classified_integrated['Most_Confident_Topic'] = ''
classified_integrated['Confidence_Score'] = ''

for index, row in merged_df.iterrows():
    if row['First_Confidence_Score_title'] >= row['First_Confidence_Score_content']:
        classified_integrated.loc[index, 'Most_Confident_Topic'] = row['Predicted_Topics_title'].split(', ')[0]
        classified_integrated.loc[index, 'Confidence_Score'] = row['First_Confidence_Score_title']
    else:
        classified_integrated.loc[index, 'Most_Confident_Topic'] = row['Predicted_Topics_content'].split(', ')[0]
        classified_integrated.loc[index, 'Confidence_Score'] = row['First_Confidence_Score_content']

# Save the new DataFrame to a CSV file
classified_integrated.to_csv('classified_integrated.csv', index=False)
print("Integrated classification results saved to 'classified_integrated.csv'")
files.download('classified_integrated.csv')

Integrated classification results saved to 'classified_integrated.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Load the classified dataframes
# Function to extract the first confidence score
def get_first_confidence_score(score_str):
    try:
        return float(score_str.split(',')[0])
    except (ValueError, IndexError, AttributeError):
        return 0  # Handle cases with missing or invalid scores

# Add first confidence scores to both DataFrames
classified_title_df['First_Confidence_Score'] = classified_title_df['Confidence_Scores'].apply(get_first_confidence_score)
classified_content_df['First_Confidence_Score'] = classified_content_df['Confidence_Scores'].apply(get_first_confidence_score)

# Merge the dataframes on the 'URL' column
merged_df = pd.merge(classified_title_df, classified_content_df, on='URL', suffixes=('_title', '_content'))

# Create a new DataFrame 'classified_integrated' with the higher confidence score and associated data
classified_integrated = pd.DataFrame()
classified_integrated['Title'] = merged_df['Title_title']  # Take titles from the title-based classification
classified_integrated['URL'] = merged_df['URL']

# Compare confidence scores and select the best prediction for each URL
classified_integrated['Predicted_Topics'] = merged_df.apply(
    lambda row: row['Predicted_Topics_title'] if row['First_Confidence_Score_title'] >= row['First_Confidence_Score_content']
    else row['Predicted_Topics_content'], axis=1
)
classified_integrated['Confidence_Scores'] = merged_df.apply(
    lambda row: row['Confidence_Scores_title'] if row['First_Confidence_Score_title'] >= row['First_Confidence_Score_content']
    else row['Confidence_Scores_content'], axis=1
)

# Save the new DataFrame to a CSV file
classified_integrated.to_csv('classified_integrated.csv', index=False)
print("Integrated classification results saved to 'classified_integrated.csv'")

# Optional: Download the file if running in Colab
try:
    from google.colab import files
    files.download('classified_integrated.csv')
except ImportError:
    print("File download skipped (not running in Google Colab).")

Integrated classification results saved to 'classified_integrated.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt: I want to compare the first confidence score for each URL in classified_title_df vs that in classified_content_df and save the topics of with a higher score. Let's call the new data frame classified_integrated. I wrote this code, but the data it creates is just a column of titles and a column of a all predicted topics added together. I want these columns for classified_integrated: title, URL, most confident topics, prediction confidence scores for those topics. Here is an example, if the a URL has "client intake" as topic with a prediction confidence score of 0.7 based on its title and it's classified under "document management" with a prediction confidence score of 0.85 based on its content, I want its topic and confidence score to be taken from classified_content (the more confident prediction). But if the difference between confidence scores are less than 0.1, I want to append the lesser score and its predicted topic to the greater one in the final integrated table.

# Load the classified dataframes
# Function to extract the first confidence score and topic
def get_first_confidence_and_topic(score_str, topic_str):
    try:
        scores = [float(s) for s in score_str.split(',')]
        topics = topic_str.split(', ')
        return scores[0], topics[0]
    except (ValueError, IndexError, AttributeError):
        return 0, "No Topic"

# Apply the function to create new columns
classified_title_df[['First_Confidence_Score', 'First_Predicted_Topic']] = classified_title_df.apply(
    lambda row: pd.Series(get_first_confidence_and_topic(row['Confidence_Scores'], row['Predicted_Topics'])), axis=1)
classified_content_df[['First_Confidence_Score', 'First_Predicted_Topic']] = classified_content_df.apply(
    lambda row: pd.Series(get_first_confidence_and_topic(row['Confidence_Scores'], row['Predicted_Topics'])), axis=1)


# Merge the dataframes
merged_df = pd.merge(classified_title_df, classified_content_df, on='URL', suffixes=('_title', '_content'))

# Create the integrated dataframe
classified_integrated = pd.DataFrame()
classified_integrated['Title'] = merged_df['Title_title']
classified_integrated['URL'] = merged_df['URL']
classified_integrated['Most_Confident_Topic'] = ''
classified_integrated['Confidence_Score'] = ''


for index, row in merged_df.iterrows():
    if abs(row['First_Confidence_Score_title'] - row['First_Confidence_Score_content']) < 0.1:
        classified_integrated.loc[index, 'Most_Confident_Topic'] = f"{row['First_Predicted_Topic_title']}, {row['First_Predicted_Topic_content']}"
        classified_integrated.loc[index, 'Confidence_Score'] = f"{row['First_Confidence_Score_title']}, {row['First_Confidence_Score_content']}"
      # if row['First_Confidence_Score_title'] > row['First_Confidence_Score_content']:
      #   classified_integrated.loc[index, 'Most_Confident_Topic'] = f"{row['First_Predicted_Topic_title']}, {row['First_Predicted_Topic_content']}"
      #   classified_integrated.loc[index, 'Confidence_Score'] = f"{row['First_Confidence_Score_title']}, {row['First_Confidence_Score_content']}"
      # else:
      #   classified_integrated.loc[index, 'Most_Confident_Topic'] = f"{row['First_Predicted_Topic_content']}, {row['First_Predicted_Topic_title']}"
      #   classified_integrated.loc[index, 'Confidence_Score'] = f"{row['First_Confidence_Score_content']}, {row['First_Confidence_Score_title']}"
    elif row['First_Confidence_Score_title'] > row['First_Confidence_Score_content']:
        classified_integrated.loc[index, 'Most_Confident_Topic'] = row['First_Predicted_Topic_title']
        classified_integrated.loc[index, 'Confidence_Score'] = row['First_Confidence_Score_title']
    else:
        classified_integrated.loc[index, 'Most_Confident_Topic'] = row['First_Predicted_Topic_content']
        classified_integrated.loc[index, 'Confidence_Score'] = row['First_Confidence_Score_content']

# Save the integrated dataframe
classified_integrated.to_csv('classified_integrated.csv', index=False)
print("Integrated classification results saved to 'classified_integrated.csv'")

# Download the file (if running in Google Colab)
try:
    files.download('classified_integrated.csv')
except ImportError:
    print("File download skipped (not running in Google Colab).")

Integrated classification results saved to 'classified_integrated.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

from googleapiclient.discovery import build

# Replace with your actual values
site_url = "https://www.example.com/"
page_url = "https://www.example.com/specific-page/"
start_date = "2023-10-26"
end_date = "2023-11-26"

service = build("searchconsole", "v1")

request = {
    "startDate": start_date,
    "endDate": end_date,
    "dimensions": ["query"],
    "dimensionFilterGroups": [
        {
            "filters": [
                {
                    "dimension": "page",
                    "operator": "equals",
                    "expression": page_url,
                }
            ]
        }
    ],
}

response = service.searchanalytics().query(siteUrl=site_url, body=request).execute()

# Extract keywords from the response
keywords = [row["keys"][0] for row in response.get("rows", [])]

print(keywords)
